Generate learning data (Euler truncation errors) from a single trajectory of the bubble equation.

In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt
import scipy.integrate
import h5py

In [ ]:
path = '../data/bubble_sim_p0.5_f100_Re10_N16_t0.5_25000.txt'

In [ ]:
data = np.loadtxt(path, dtype=np.float64, delimiter=',')
data.shape

In [ ]:
plt.figure(num="vis_dat")
plt.subplot(4,1,1)
plt.plot(data[:,0], data[:,1])
plt.subplot(4,1,2)
plt.plot(data[:,0], data[:,2])
plt.subplot(4,1,3)
plt.plot(data[:,0], data[:,3])
plt.subplot(4,1,4)
plt.plot(data[:,0], data[:,11])
plt.show()

In [ ]:
def euler_truncation_error(arr, output_size): 
    #t0 x1 x2 x3 z1 ... z7 dx1 dx2 dx3 dz1 ... dz7 sin
    #0   1  2  3 4       10 11  12  13  14      20  21
    dt = arr[1:,0] - arr[:-1,0] #next-prev
    X = np.column_stack((arr[1:,0], arr[:-1,:1+output_size], arr[:-1,-1])) #t1 t0 x1(0) x2(0) x3(0) z(0) sin0
    dt_m = np.copy(dt)
    for n in range(1,output_size):
        dt_m = np.column_stack((dt_m,dt))
    Y = np.reciprocal(dt_m*dt_m)*(arr[1:,1:output_size+1] - arr[:-1,1:output_size+1] - dt_m*arr[:-1, output_size+1:-1])
    return X,Y

In [ ]:
path_to_hdf = '../data/data0.5.hdf5'

In [ ]:
data = np.loadtxt(path, dtype=np.float64, delimiter=',')
arr = np.delete(data, [4, 15], axis=1) #remove boundary
print(arr.shape)
print(arr[5,:])
N_z = int((data.shape[1]-8)/2 - 1)


dt = True #whether to use absolute time or time steps
l = arr.shape[0]
b = 1
n = 100
sum = 0
for i in range(b,n):
    sum = sum + l - i

print(sum)
with h5py.File(path_to_hdf, 'a') as f:
    f.create_dataset(
        str('bub_X'),
        (sum, 5+N_z),
        dtype   = np.float64,
        compression     = 'gzip',
        compression_opts= 9
        )
    f.create_dataset(
        str('bub_Y'),
        (sum, 3+N_z),
        dtype   = np.float64,
        compression     = 'gzip',
        compression_opts= 9
        )
    begin = 0
    end = l-1
    X = f['bub_X']
    Y = f['bub_Y']
    x,y = euler_truncation_error(np.copy(arr[:][0::b]),3+N_z)
    if dt: 
        x = np.column_stack((x[:,0] - x[:,1],x[:,2:]))
    X[begin:end,:] = x
    Y[begin:end,:] = y
    for i in range(b+1,n):
        if i%50==0:
            print(i)
        for j in range(i):
            x,y = euler_truncation_error(np.copy(arr[:][j::i]), 3+N_z)
            if dt: 
                x = np.column_stack((x[:,0] - x[:,1],x[:,2:]))
            begin = end
            end = begin+x.shape[0]
            X[begin:end,:] = x
            Y[begin:end,:] = y
        